In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
f = open('finitefault.txt')
lines = f.readlines()
lines[106] = lines[106].replace('Hypo','')
data = [[ float(s) for s in line.split()] for line in lines[27:]]

In [3]:
import pandas as pd
df = pd.DataFrame(data, columns = 'lat lon z rake slip ontime'.split())
df = df.rename(columns={'z':'depth'})
df['depth'] = 1000*df['depth']
df['strike'] = 15
df['dip'] = 18
df['L'] = 40000
df['W'] = 40000
df['reference'] = 'center'
df['slip'] /= 100
df['lon'] +=360
df.head()

,lat,lon,depth,rake,slip,ontime,strike,dip,L,W,reference
0,-39.001,287.862,81082.0,111.0,0.43,142.5,15,18,40000,40000,center
1,-38.652,287.977,81082.0,102.0,0.91,122.3,15,18,40000,40000,center
2,-38.304,288.092,81082.0,112.1,0.03,106.2,15,18,40000,40000,center
3,-37.956,288.206,81082.0,111.9,0.03,92.1,15,18,40000,40000,center
4,-37.608,288.321,81082.0,114.0,1.14,79.0,15,18,40000,40000,center


In [4]:
df = df['lat lon depth slip strike dip rake L W reference'.split()]

In [5]:
df.to_csv('earthquake.csv',index=False)

# Easywave format

In [6]:
df['reference']='C'
df['depth'] /=1000
df['L'] /=1000
df['W'] /=1000

In [7]:
f = open('fault.flt','w')

for index, row in df.iterrows():
    rowString = '-location {lon} {lat} {depth} -refpos {reference} -strike {strike} -dip {dip} -rake {rake} -slip {slip} -size {L} {W}\n'
    rowString = rowString.format(**row)
    f.write(rowString)

f.close()